<a href="https://www.kaggle.com/code/bugrayildirim/brain-tumor-mri-classification?scriptVersionId=264003414" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Akbank Derin Öğrenme Bootcamp: Brain Tumor MRI Classification
**Proje amacı:** CNN tabanlı model kullanarak MR görüntülerinden beyin tümörünü sınıflandırmak; model performansını değerlendirmek ve Grad-CAM ile model açıklanabilirliği sağlamak.

**Veri seti:** masoudnickparvar/brain-tumor-mri-dataset (Kaggle). Dataset 4 sınıf içerir: glioma, meningioma, pituitary, no_tumor (toplam ~7k görüntü).

**Geliştirme ortamı:** Kaggle Notebook (tüm hücreler ve görselleştirmeler notebook içinde yer alacaktır). Bootcamp gereksinimleri dikkate alındı.


In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, callbacks

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Kaggle dataset root 
DATA_DIR = "/kaggle/input/brain-tumor-mri-dataset"
print("DATA_DIR exists:", os.path.exists(DATA_DIR))


## 1. Veri Ön İşleme ve Data Augmentation

- Görüntüler `flow_from_directory` ile sınıf klasörlerinden okunacak.
- Eğitim sırasında ImageDataGenerator ile dönüşümler uygulanacak: rotation, horizontal/vertical flip, zoom, brightness (color jitter etkisi).
- Train/validation/test split: dataset klasörleri hazır bölünmemişse `validation_split` ile split yapılacaktır.


In [ ]:
# Parametreler
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.8,1.2],
    validation_split=0.2,   # train/val split
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = os.path.join(DATA_DIR, "Training") if os.path.exists(os.path.join(DATA_DIR, "Training")) else DATA_DIR
# Eğer dataset doğrudan class klasörleri içeriyorsa DATA_DIR kullan; değilse Kaggle versiyonuna göre "Training" veya "train" klasörlerini kontrol et.
print("Listing top-level:", os.listdir(DATA_DIR)[:10])

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    seed=SEED
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    seed=SEED
)

# Eğer ayrı test klasörü varsa onu da test_datagen ile yükleyin:
test_dir = os.path.join(DATA_DIR, "Testing")
if os.path.exists(test_dir):
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )
else:
    test_generator = None


## 2. Model Mimarisi ve Eğitimi

İki seçenek sunuyorum:
- A) Küçük, baştan inşa edilmiş bir CNN (örnek: baseline).
- B) Transfer Learning: Önceden eğitilmiş bir model (ör: EfficientNetB0 veya VGG16) üzerine fine-tune.

Aşağıda önce baseline CNN kodu veriliyor; ardından transfer learning örneği de yorum satırıyla eklenmiştir.


In [ ]:
num_classes = train_generator.num_classes
print("Num classes:", num_classes)

def build_baseline_cnn(input_shape=IMG_SIZE + (3,), num_classes=num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs, name="baseline_cnn")
    return model

model = build_baseline_cnn()
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()


## 3. Transfer Learning ile Model Geliştirme (Opsiyonel)

Burada, önceden eğitilmiş bir model (EfficientNetB0) kullanarak fine-tuning yapacağız.  
Bu yaklaşım sayesinde sınırlı veriyle daha güçlü özellik çıkarımı sağlanabilir.  
İlk aşamada base model eğitimde **dondurulacak**, daha sonra bazı katmanlar eğitim moduna alınabilir.


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False  # önce feature extractor olarak kullan

inputs = layers.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
tl_model = models.Model(inputs, outputs)

tl_model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
tl_model.summary()


## 4. Model Eğitimi: Callbacks & Eğitim Süreci

Aşağıda kullanılan callback’ler:
- **ModelCheckpoint**: En iyi modeli kaydetme  
- **EarlyStopping**: Aşırı öğrenmeyi önlemek için durdurma  
- **ReduceLROnPlateau**: Validation loss iyileşmezse learning rate’i düşürme  

Eğitim epoch sayısı ve callback yapılandırması aşağıdadır.


In [ ]:
EPOCHS = 15

checkpoint_cb = callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", mode="min")
earlystopping_cb = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reduce_lr_cb = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint_cb, earlystopping_cb, reduce_lr_cb]
)


## 5. Eğitim Sonuçları: Accuracy & Loss Grafikleri

Bu hücre ile eğitim sürecindeki doğruluk (accuracy) ve kayıp (loss) eğilimlerini görselleştireceğiz.  
Grafikler, overfitting / underfitting durumlarını izlemede yardımcı olur.


In [ ]:
# Eğitim grafikleri
def plot_history(h):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(h.history['accuracy'], label='train_acc')
    plt.plot(h.history['val_accuracy'], label='val_acc')
    plt.title('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1,2,2)
    plt.plot(h.history['loss'], label='train_loss')
    plt.plot(h.history['val_loss'], label='val_loss')
    plt.title('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

plot_history(history)


## 6. Model Değerlendirme: Confusion Matrix & Classification Report

Test veya validation seti üzerinde modelin sınıflandırma performansını inceleyeceğiz.  
- **Classification Report** ile precision, recall, f1-score değerlerini göreceğiz  
- **Confusion Matrix** ile sınıflar arasındaki karışmaları görselleştireceğiz  


In [ ]:
# Eğer test_generator varsa kullan; yoksa validation üzerinde değerlendirme yap
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

if test_generator is not None:
    steps = test_generator.samples // test_generator.batch_size + 1
    preds = model.predict(test_generator, steps=steps)
    y_pred = np.argmax(preds, axis=1)
    y_true = test_generator.classes
    class_indices = test_generator.class_indices
else:
    # validation generator üzerinden tahmin örneği
    steps = validation_generator.samples // validation_generator.batch_size + 1
    preds = model.predict(validation_generator, steps=steps)
    y_pred = np.argmax(preds, axis=1)
    y_true = validation_generator.classes
    class_indices = validation_generator.class_indices

labels = list(class_indices.keys())
print("Classes:", labels)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix')
plt.show()


## 7. Grad-CAM Görselleştirmesi

Grad-CAM ile modelin hangi bölgelere bakarak karar verdiğini görselleştiriyoruz. Keras örnekleri ve rehberleriyle uyumlu bir uygulama takip edilmektedir. (Keras Grad-CAM örneği). :contentReference[oaicite:7]{index=7}


In [ ]:
# Grad-CAM fonksiyonu — Keras resmi örneğinden uyarlanmıştır. :contentReference[oaicite:8]{index=8}
import tensorflow.keras.backend as K
import cv2

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-10)
    return heatmap.numpy()

# Örnek kullanım: validation'dan bir batch al ve Grad-CAM uygula
last_conv_layer = None
# otomatik bulmaya çalış
for layer in reversed(model.layers):
    if isinstance(layer, layers.Conv2D):
        last_conv_layer = layer.name
        break
print("Last conv layer:", last_conv_layer)

# Test için tek bir görüntü al
x_batch, y_batch = next(iter(validation_generator))
img = x_batch[0]
inp = np.expand_dims(img, axis=0)

heatmap = make_gradcam_heatmap(inp, model, last_conv_layer)
# Resize heatmap to image size
heatmap = cv2.resize(heatmap, (IMG_SIZE[1], IMG_SIZE[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(cv2.cvtColor((img*255).astype('uint8'), cv2.COLOR_RGB2BGR), 0.6, heatmap, 0.4, 0)
# Display
plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(img); plt.title('Input'); plt.axis('off')
plt.subplot(1,2,2); plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)); plt.title('Grad-CAM'); plt.axis('off')
plt.show()


## 8. Hiperparametre Optimizasyonu

Deney yapılan parametreler:
- Katman & filtre sayısı: 32→64→128 gibi artışlar
- Kernel boyutları: 3x3 standardı, bazı yerlerde 5x5 denemesi
- Dropout: 0.2 - 0.5 aralığı
- Learning rate: 1e-3 → 1e-5 arası denemeler
- Batch size: 16, 32, 64
- Optimizer: Adam, SGD+momentum

Otomatik arama için Keras Tuner veya RandomSearch önerilir.


## 9. Sonuçlar ve Değerlendirme

- Model: Baseline CNN (ve/veya EfficientNetB0 transfer learning).
- En iyi validation accuracy: **(buraya deneysel sonuçlarınızı yazın)**.
- Confusion matrix ve classification report yukarıda gösterildi.
- Grad-CAM görselleştirmeleri, modelin görüntüdeki tümör bölgelerine odaklandığını doğruladı / doğrulamadı (bunu gözle kontrol ederek yorumlayın).

**Gelecek çalışmalar:**
- Daha güçlü transfer learning (EfficientNet / ResNet50) ve fine-tuning
- Class imbalance varsa class-weight veya oversampling uygulama
- Segmentation (U-Net) ile lokasyon tespiti ve ardından classification
